In [1]:
#enveloping potential
# simple Example plot Enveloped Potential with two Harmonic Oscilators
##Imports:
import os, sys as csys, datetime, tempfile
import math, numpy as np, pandas as pd
import scipy.constants as const

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


csys.path.append(os.getcwd()+"/../../..")

#Ensembler
import ensembler.potentials.TwoD as pot2D

from ensembler.samplers.stochastic import metropolisMonteCarloIntegrator
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.system.basic_system import system

import ensembler.visualisation.plotPotentials as exPlot

%matplotlib inline

In [2]:
#ENERGIES Sampling
def write_out_etraj(traj, out_path, V1, V2):
    visited_positions =  np.array(list(map(np.array, traj.position)))
    state1 = V1.ene(visited_positions)
    state2 = V2.ene(visited_positions)
    Vrenergies = np.array(traj.total_potential_energy)

    out_file = open(out_path, "w")
    out_file.write("t\tV1\tV2\tVr\n")#HEADER
    for t, V1, V2, Vr in zip(traj.index, state1, state2, Vrenergies):
        out_file.write(str(t)+"\t"+str(V1)+"\t"+str(V2)+"\t"+str(Vr)+"\n")

    out_file.close()

In [3]:
    
#METROPOLIS CRITERION
##random part of Metropolis Criterion:
_defaultRandomness = lambda V1, V2: (np.random.rand() <= np.exp(-1.0 / (const.gas_constant / 1000.0 * 298) * (V1 - V2))) #pseudocount  for equal energies
##default Metropolis Criterion
_defaultMetropolisCriterion = lambda V1, V2: (V1 < V2 or _defaultRandomness(V1, V2))

    
def MC_exchange(sys1, sys2):
    s1 = sys1.potential.s
    s2 = sys2.potential.s

    V1_s1 = sys1.potential.ene(sys1._currentPosition)

    V2_s2 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s2
    V1_s2 = sys1.potential.ene(sys1._currentPosition)

    sys2.potential.s = s1
    V2_s1 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s1
    sys2.potential.s = s2

    exchange = _defaultMetropolisCriterion((V2_s1+V1_s2),(V1_s1+V2_s2))
    return exchange

In [4]:
#Simulation Setup
##Build System
periodic_bound = periodicBoundaryCondition(boundary=[[-180,180], [-180,180]])
integrator = metropolisMonteCarloIntegrator(fixed_step_size=[np.deg2rad(1),np.deg2rad(1)], max_iteration_tillAccept=1000)


##STEPS
replicas = 10
each_sim = 20#0 #steps between trials
trials = 10#00  #exchange attempts

#s_est = 0.609/Delta U_barrier
optimal_ses = {2.5: 0.122,
               5: 0.061,
               10: 0.03,
               20: 0.03, #0.015
               50: 0.006}

initial_position = None



In [ ]:
#run multiple replicas
root_tmp_dir = tempfile.gettempdir()+"/edsSim"
if(not os.path.exists(root_tmp_dir)):
    os.mkdir(root_tmp_dir)
os.chdir(root_tmp_dir)

tmp_dir = root_tmp_dir+"/re_eds_simulation"
if(not os.path.exists(tmp_dir)):
    os.mkdir(tmp_dir)
os.chdir(tmp_dir)

print(tmp_dir)


for barrier in tqdm(optimal_ses, desc="Barrier: "):
    approach_dir = tmp_dir+"/simulation_with_"+str(barrier)+"kt_barrier"
    if(not os.path.exists(approach_dir)): os.mkdir(approach_dir)


    start_t = datetime.datetime.now()
    
    #build potential
    amplitude = barrier/2
    shift  = np.rad2deg(0.5*math.pi)
    Eoff = [0,0]

    V1 = pot2D.wavePotential(phase_shift=(shift,shift), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    V2 = pot2D.wavePotential(phase_shift=(0.0, 0.0), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    edsPot1 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    edsPot2 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    
    
    start=0
    syst = None
    for replica in tqdm(range(start, replicas), desc="Replicas: "):
        ##INIT REPLICA
        ###fILE hANDLING
        replica_out = approach_dir+"/replica_"+str(replica)
        if(not os.path.exists(replica_out)):
            os.mkdir(replica_out)

        ###Build System
        sys1=system(potential=edsPot1, sampler=integrator, conditions=[periodic_bound])
        sys2=system(potential=edsPot2, sampler=integrator, conditions=[periodic_bound])

        sys1.potential.s = 0.5
        sys2.potential.s = 0.01

        ## SIMULATE REPLICA
        start_sim = datetime.datetime.now()
        s_val_posDict = {}  #OUT
        current_s_index = 0
        current_s_direction = 1

        for trial in tqdm(range(trials), desc="Current Simulation of replica "+str(replica)+": ",):
            cur_state1 = sys1.simulate(each_sim, withdraw_traj=False, verbosity=False)
            cur_state2 = sys2.simulate(each_sim, withdraw_traj=False, verbosity=False)

            if(MC_exchange(sys1, sys2)):
                sys1._currentPosition = cur_state2.position
                sys2._currentPosition = cur_state1.position
            else:
                pass

        s_val_posDict.update({sys2.potential.s[0] : sys2.trajectory})
        s_val_posDict.update({sys1.potential.s[0] : sys1.trajectory})

        end_sim = datetime.datetime.now()
        duration_sim = end_sim-start_sim

        #plotting:
        print("ploting")
        start_plot = datetime.datetime.now()
        fig = exPlot.plot_2D_2State_EDS_potential_sDependency(s_val_posDict, eds_pot=edsPot1, plot_trajs=True, out_path=replica_out+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT_withTraj.png")
        end_plot = datetime.datetime.now()
        duration_plot = end_plot-start_plot

        for s in s_val_posDict:
            write_out_etraj(traj=s_val_posDict[s], out_path=replica_out+"/replica_traj_s"+str(s)+".dat", V1=sys1.potential.V_is[0], V2=sys1.potential.V_is[1])
        del sys1, sys2



    end_t = datetime.datetime.now()
    duration = end_t-start_t
    print("Done - duration: ", duration)
    print("Done - simulation duration: ", duration_sim)
    print("Done - plotting duration: ", duration_plot)

print(root_tmp_dir)

C:\Users\benja\AppData\Local\Temp/edsSim/re_eds_simulation



ploting
[-6.85542006 -6.83796677 -6.85542006 -6.83796677 -6.82051348 -6.80306019
 -6.78560689 -6.80306019 -6.82051348 -6.83796677 -6.85542006 -6.87287336
 -6.85542006 -6.87287336 -6.89032665 -6.90777994 -6.92523323 -6.94268653
 -6.96013982 -6.97759311 -6.9950464   6.9248371   6.94229039  6.9248371
  6.90738381  6.9248371   6.94229039  6.95974368  6.94229039  6.95974368
  6.94229039  6.9248371   6.94229039  6.9248371   6.94229039  6.95974368
  6.94229039  6.95974368  6.97719698  6.95974368  6.97719698 -6.83796677
 -6.82051348 -6.80306019 -6.82051348 -6.80306019 -6.78560689 -6.7681536
 -6.78560689 -6.80306019 -6.78560689 -6.80306019 -6.82051348 -6.80306019
 -6.82051348 -6.83796677 -6.82051348 -6.80306019 -6.82051348 -6.83796677
 -6.85542006  6.99465027  7.01210356  7.02955685  7.01210356  7.02955685
  7.04701015  7.06446344  7.04701015  7.02955685  7.04701015  7.02955685
  7.04701015  7.06446344  7.08191673  7.06446344  7.04701015  7.02955685
  7.04701015  7.06446344  7.04701015 -6.9077

 -1.01420666 -0.99675337 -1.01420666]



ploting
[-1.25298941 -1.27044271 -1.287896   -1.30534929 -1.32280258 -1.34025588
 -1.35770917 -1.37516246 -1.35770917 -1.34025588 -1.32280258 -1.34025588
 -1.32280258 -1.34025588 -1.32280258 -1.34025588 -1.35770917 -1.37516246
 -1.35770917 -1.37516246 -1.39261575 -5.82854489 -5.8110916  -5.79363831
 -5.77618502 -5.75873172 -5.77618502 -5.75873172 -5.77618502 -5.79363831
 -5.77618502 -5.79363831 -5.8110916  -5.79363831 -5.8110916  -5.79363831
 -5.8110916  -5.79363831 -5.77618502 -5.79363831 -5.77618502 -1.41006905
 -1.39261575 -1.41006905 -1.42752234 -1.44497563 -1.42752234 -1.44497563
 -1.46242892 -1.47988222 -1.46242892 -1.44497563 -1.42752234 -1.44497563
 -1.46242892 -1.44497563 -1.42752234 -1.41006905 -1.42752234 -1.41006905
 -1.42752234 -5.68891855 -5.67146526 -5.65401197 -5.63655868 -5.65401197
 -5.63655868 -5.61910538 -5.63655868 -5.65401197 -5.63655868 -5.65401197
 -5.63655868 -5.65401197 -5.63655868 -5.61910538 -5.60165209 -5.61910538
 -5.63655868 -5.65401197 -5.67146526 -1.51


ploting
[ 0.36365336  0.34620007  0.36365336  0.38110666  0.36365336  0.38110666
  0.39855995  0.38110666  0.36365336  0.38110666  0.39855995  0.38110666
  0.36365336  0.34620007  0.32874678  0.34620007  0.36365336  0.34620007
  0.36365336  0.38110666  0.39855995 -0.50147802 -0.51893131 -0.53638461
 -0.5538379  -0.57129119 -0.58874448 -0.60619778 -0.62365107 -0.64110436
 -0.62365107 -0.64110436 -0.62365107 -0.60619778 -0.62365107 -0.64110436
 -0.65855765 -0.67601095 -0.69346424 -0.67601095 -0.65855765  0.59054617
  0.57309287  0.59054617  0.57309287  0.59054617  0.60799946  0.59054617
  0.57309287  0.55563958  0.53818629  0.55563958  0.57309287  0.55563958
  0.53818629  0.55563958  0.53818629  0.55563958  0.57309287  0.55563958
  0.53818629 -0.81563729 -0.79818399 -0.81563729 -0.83309058 -0.81563729
 -0.79818399 -0.81563729 -0.83309058 -0.81563729 -0.79818399 -0.81563729
 -0.83309058 -0.85054387 -0.86799716 -0.85054387 -0.86799716 -0.88545046
 -0.86799716 -0.85054387 -0.83309058  0.52

...


ploting
[ 9.66760195  9.65014865  9.66760195  9.65014865  9.66760195  9.65014865
  9.63269536  9.65014865  9.63269536  9.65014865  9.63269536  9.65014865
  9.66760195  9.65014865  9.66760195  9.68505524  9.66760195  9.65014865
  9.63269536  9.65014865  9.63269536 -8.01338714 -8.03084044 -8.01338714
 -7.99593385 -7.97848056 -7.99593385 -7.97848056 -7.96102727 -7.94357397
 -7.96102727 -7.94357397 -7.96102727 -7.97848056 -7.99593385 -8.01338714
 -7.99593385 -7.97848056 -7.96102727 -7.94357397 -7.96102727  9.65014865
  9.66760195  9.68505524  9.70250853  9.71996182  9.70250853  9.68505524
  9.70250853  9.71996182  9.70250853  9.71996182  9.73741512  9.71996182
  9.70250853  9.68505524  9.66760195  9.68505524  9.66760195  9.65014865
  9.66760195 -8.01338714 -8.03084044 -8.01338714 -8.03084044 -8.01338714
 -8.03084044 -8.04829373 -8.03084044 -8.04829373 -8.03084044 -8.01338714
 -8.03084044 -8.01338714 -7.99593385 -7.97848056 -7.99593385 -7.97848056
 -7.99593385 -7.97848056 -7.99593385  9.71


ploting
[ -9.98891029  -9.97145699  -9.9540037   -9.93655041  -9.9540037
  -9.93655041  -9.9540037   -9.97145699  -9.98891029 -10.00636358
  -9.98891029  -9.97145699  -9.98891029 -10.00636358 -10.02381687
 -10.00636358 -10.02381687 -10.00636358 -10.02381687 -10.00636358
 -10.02381687   2.89974099   2.8822877    2.89974099   2.8822877
   2.86483441   2.8822877    2.89974099   2.8822877    2.86483441
   2.8822877    2.86483441   2.84738111   2.82992782   2.84738111
   2.82992782   2.84738111   2.86483441   2.84738111   2.82992782
   2.84738111  -9.93655041  -9.91909712  -9.90164382  -9.91909712
  -9.90164382  -9.88419053  -9.86673724  -9.88419053  -9.90164382
  -9.88419053  -9.86673724  -9.84928395  -9.83183065  -9.81437736
  -9.79692407  -9.77947078  -9.76201748  -9.77947078  -9.76201748
  -9.77947078   2.72520807   2.70775477   2.69030148   2.67284819
   2.6553949    2.6379416    2.62048831   2.60303502   2.62048831
   2.60303502   2.58558173   2.60303502   2.58558173   2.56812843
   


ploting
[-8.1774358  -8.15998251 -8.14252922 -8.15998251 -8.14252922 -8.12507592
 -8.10762263 -8.12507592 -8.10762263 -8.12507592 -8.10762263 -8.12507592
 -8.14252922 -8.12507592 -8.10762263 -8.09016934 -8.10762263 -8.12507592
 -8.10762263 -8.09016934 -8.07271605  6.9047551   6.92220839  6.93966168
  6.95711497  6.93966168  6.92220839  6.9047551   6.8873018   6.9047551
  6.92220839  6.93966168  6.95711497  6.93966168  6.92220839  6.93966168
  6.95711497  6.97456827  6.95711497  6.97456827  6.99202156 -8.05526275
 -8.03780946 -8.05526275 -8.07271605 -8.09016934 -8.10762263 -8.09016934
 -8.07271605 -8.05526275 -8.07271605 -8.09016934 -8.07271605 -8.09016934
 -8.10762263 -8.09016934 -8.07271605 -8.09016934 -8.07271605 -8.05526275
 -8.07271605  7.00947485  6.99202156  7.00947485  7.02692814  7.04438144
  7.02692814  7.04438144  7.06183473  7.07928802  7.06183473  7.07928802
  7.09674131  7.07928802  7.06183473  7.07928802  7.06183473  7.04438144
  7.06183473  7.07928802  7.09674131 -7.985


ploting
[ 6.15586766  6.17332095  6.15586766  6.17332095  6.15586766  6.17332095
  6.15586766  6.17332095  6.19077425  6.17332095  6.15586766  6.17332095
  6.19077425  6.17332095  6.19077425  6.20822754  6.19077425  6.20822754
  6.19077425  6.17332095  6.19077425 -4.94126455 -4.92381125 -4.90635796
 -4.92381125 -4.94126455 -4.92381125 -4.90635796 -4.88890467 -4.87145138
 -4.88890467 -4.90635796 -4.92381125 -4.94126455 -4.92381125 -4.90635796
 -4.92381125 -4.90635796 -4.88890467 -4.87145138 -4.88890467  6.20822754
  6.22568083  6.24313412  6.22568083  6.20822754  6.19077425  6.20822754
  6.22568083  6.24313412  6.26058742  6.27804071  6.295494    6.27804071
  6.26058742  6.24313412  6.26058742  6.27804071  6.295494    6.27804071
  6.295494   -4.90635796 -4.92381125 -4.90635796 -4.88890467 -4.87145138
 -4.85399808 -4.87145138 -4.88890467 -4.90635796 -4.88890467 -4.90635796
 -4.88890467 -4.90635796 -4.92381125 -4.94126455 -4.95871784 -4.97617113
 -4.99362442 -5.01107772 -5.02853101  6.27


ploting
[7.81316501 7.79571171 7.77825842 7.79571171 7.81316501 7.8306183
 7.81316501 7.8306183  7.84807159 7.86552488 7.84807159 7.8306183
 7.84807159 7.86552488 7.88297818 7.90043147 7.88297818 7.86552488
 7.84807159 7.8306183  7.81316501 1.4795863  1.49703959 1.51449288
 1.53194618 1.51449288 1.49703959 1.4795863  1.46213301 1.4795863
 1.46213301 1.44467971 1.46213301 1.44467971 1.42722642 1.40977313
 1.39231984 1.37486654 1.35741325 1.33995996 1.35741325 7.76080513
 7.74335184 7.76080513 7.77825842 7.79571171 7.77825842 7.76080513
 7.77825842 7.79571171 7.81316501 7.79571171 7.81316501 7.8306183
 7.84807159 7.86552488 7.84807159 7.86552488 7.84807159 7.86552488
 7.88297818 1.44467971 1.46213301 1.44467971 1.46213301 1.44467971
 1.42722642 1.40977313 1.42722642 1.40977313 1.42722642 1.44467971
 1.46213301 1.4795863  1.46213301 1.4795863  1.46213301 1.44467971
 1.42722642 1.40977313 1.39231984 7.90043147 7.91788476 7.90043147
 7.88297818 7.90043147 7.91788476 7.90043147 7.88297818 7

In [ ]:
print(root_tmp_dir)


In [ ]:
edsPot1.constants[edsPot1.nDimensions]

In [ ]:
edsPot1.s

In [ ]:
tmp_dir